In [25]:
import os
import glob
from pathlib import Path
from typing import List

import av
import numpy as np
import torch
from moviepy import editor
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pytorchvideo.data.encoded_video import EncodedVideo
from transformers import (
    VideoMAEForVideoClassification,
    VideoMAEImageProcessor,
    VideoMAEModel,
)

np.random.seed(0)

## **Helper Functions**

In [26]:
# Got them from: https://huggingface.co/docs/transformers/model_doc/videomae#transformers.VideoMAEForVideoClassification.forward.example


def read_video_pyav(container, indices):
    """
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    """
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [33]:
def split_video_to_clips(video_path: str, output_folder: str, window: int) -> None:
    # Load video and get its duration.
    clip = editor.VideoFileClip(video_path)
    duration = clip.duration
    # del clip

    start_ = 0
    end_ = window

    os.makedirs(output_folder, exist_ok=True)

    while end_ < duration:
        # new_clip = clip.subclip(start_, end_)
        # new_clip.write_videofile(os.path.join(output_folder, f"{start_}_{end_}.mp4"), threads=1, preset='ultrafast')

        ffmpeg_extract_subclip(
            video_path,
            start_,
            end_,
            targetname=os.path.join(output_folder, f"{start_}_{end_}.mp4"),
        )
        start_ += window
        end_ += window  # TODO: There is a possibility to miss the last clip. Maybe we should set an upper bound.

## **Split a video into clips**

In [35]:
video_path = "test_video.mp4"
clips_output_path = os.path.basename(video_path)
clips_output_path = Path(clips_output_path).stem

split_video_to_clips(video_path=video_path, output_folder=clips_output_path, window=10)

## **Extract audio from a clip**

In [5]:
#Load the Video
video = editor.VideoFileClip("test_video.mp4")

#Extract the Audio
audio = video.audio

#Export the Audio
audio.write_audiofile("audio.mp3")

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


## **Extract a feature vector from a video**

In [36]:
av.open("test_video.mp4")

<av.InputContainer 'test_video.mp4'>

In [78]:
def open_clips(clips_output_path: str) -> List[List]:
    file_paths = glob.glob(os.path.join(clips_output_path, "*"))
    clips = [av.open(file_path) for file_path in file_paths]

    # sample 16 frames
    indices = [sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=clip.streams.video[0].frames) for clip in clips]
    videos = [list(read_video_pyav(clip, indices_)) for clip, indices_ in zip(clips, indices)]
    return videos

In [79]:
processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
# model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

Some weights of the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics were not used when initializing VideoMAEModel: ['fc_norm.weight', 'classifier.bias', 'fc_norm.bias', 'classifier.weight']
- This IS expected if you are initializing VideoMAEModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoMAEModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [80]:
videos = open_clips(clips_output_path=clips_output_path)

# prepare videos for the model
inputs = processor(videos, return_tensors="pt")

In [81]:
with torch.no_grad():
    outputs = model(**inputs)
    # logits = outputs.logits

In [82]:
outputs.last_hidden_state.shape

torch.Size([12, 1568, 768])

In [83]:
if len(videos) > 1:
    video_features = outputs.last_hidden_state[:, 0, :]
else:
    video_features = outputs.last_hidden_state.squeeze(dim=0)[0, :]

In [84]:
video_features.shape

torch.Size([12, 768])